## 1. Data Collection and Cleaning

### 1.1 Import Libraries and Packages

In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob, Word
from imblearn.over_sampling import ADASYN
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from datetime import datetime
from sklearn.svm import SVC
import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lookups import Lookups
from tqdm import tqdm

%matplotlib inline

### 1.2 Load Metacritic Album Review Data

__This data set consists of user reviews that were web scraped from the albums listed on www.metacritic.com. Please refer to the "Metacritic Scraper" notebook in this respository for the code used to collect this data.__



In [16]:
df = pd.read_csv('user_album_reviews.csv', engine='python')

In [17]:
df.head()

,Unnamed: 0,Unnamed: 0.1,title,artist,label,release_date,metascore,user_score,genre,summary,name,date,rating,review
0,0,0,\n\nAgricultural Tragic\n\n,Corb Lund,New West,"Jun 26, 2020",80.0,\ntbd\n,Country,The latest full-length release for the Canadia...,\nKrishnaKniar\n,"Jul 28, 2020",7.0,\nThis album has a very good vibe and you woul...
1,1,1,\n\nCloser Than Together\n\n,The Avett Brothers,Universal,"Oct 4, 2019",56.0,\ntbd\n,Folk,the 10th full-length studio release for the fo...,\ndjbrate\n,"Nov 12, 2019",1.0,\nIncredibly disappointed with the political r...
2,2,2,\n\nIII\n\n,The Lumineers,Dualtone Music,"Sep 13, 2019",72.0,\n8.6\n,Country,The third full-length release for the Colorado...,\nDididi\n,"Oct 9, 2019",10.0,"\nA literal masterpiece, its so good, very goo..."
3,3,3,\n\nIII\n\n,The Lumineers,Dualtone Music,"Sep 13, 2019",72.0,\n8.6\n,Country,The third full-length release for the Colorado...,\nbrrunosouzza\n,"Sep 25, 2019",10.0,'III' é um dos poucos álbuns que fiquei ansios...
4,4,4,\n\nIII\n\n,The Lumineers,Dualtone Music,"Sep 13, 2019",72.0,\n8.6\n,Country,The third full-length release for the Colorado...,\ngollygee93\n,"Sep 26, 2019",9.0,Rather than continuing down the path of sample...


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84536 entries, 0 to 84535
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    84536 non-null  int64  
 1   Unnamed: 0.1  84536 non-null  object 
 2   title         84054 non-null  object 
 3   artist        84054 non-null  object 
 4   label         83678 non-null  object 
 5   release_date  84054 non-null  object 
 6   metascore     84054 non-null  float64
 7   user_score    84054 non-null  object 
 8   genre         84054 non-null  object 
 9   summary       82742 non-null  object 
 10  name          84054 non-null  object 
 11  date          84054 non-null  object 
 12  rating        84054 non-null  float64
 13  review        84054 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 9.0+ MB


### 1.3 Data Cleaning

Due to the webscraping process, there are about 500 rows where reviews that had spaces in between paragraphs were made into there own row for each additional paragraph and are located in the 'Unnamed' columns. The remaining columns for those rows were all null so they will be dropped. 

In [19]:
df

,Unnamed: 0,Unnamed: 0.1,title,artist,label,release_date,metascore,user_score,genre,summary,name,date,rating,review
0,0,0,\n\nAgricultural Tragic\n\n,Corb Lund,New West,"Jun 26, 2020",80.0,\ntbd\n,Country,The latest full-length release for the Canadia...,\nKrishnaKniar\n,"Jul 28, 2020",7.0,\nThis album has a very good vibe and you woul...
1,1,1,\n\nCloser Than Together\n\n,The Avett Brothers,Universal,"Oct 4, 2019",56.0,\ntbd\n,Folk,the 10th full-length studio release for the fo...,\ndjbrate\n,"Nov 12, 2019",1.0,\nIncredibly disappointed with the political r...
2,2,2,\n\nIII\n\n,The Lumineers,Dualtone Music,"Sep 13, 2019",72.0,\n8.6\n,Country,The third full-length release for the Colorado...,\nDididi\n,"Oct 9, 2019",10.0,"\nA literal masterpiece, its so good, very goo..."
3,3,3,\n\nIII\n\n,The Lumineers,Dualtone Music,"Sep 13, 2019",72.0,\n8.6\n,Country,The third full-length release for the Colorado...,\nbrrunosouzza\n,"Sep 25, 2019",10.0,'III' é um dos poucos álbuns que fiquei ansios...
4,4,4,\n\nIII\n\n,The Lumineers,Dualtone Music,"Sep 13, 2019",72.0,\n8.6\n,Country,The third full-length release for the Colorado...,\ngollygee93\n,"Sep 26, 2019",9.0,Rather than continuing down the path of sample...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84531,10292,2,\n\nIn A Beautiful Place Out In The Country [E...,Boards of Canada,Warp,"Nov 28, 2000",78.0,\n8.9\n,Electronic,This four-track EP is the first new material f...,\nKasperG\n,"Jun 13, 2005",10.0,"For an EP, it is unusual to contain such conc..."
84532,10293,3,\n\nIn A Beautiful Place Out In The Country [E...,Boards of Canada,Warp,"Nov 28, 2000",78.0,\n8.9\n,Electronic,This four-track EP is the first new material f...,\nIsaac_P\n,"Oct 16, 2015",10.0,\nThis short EP is definitely one of those unk...
84533,10294,0,\n\nBeaucoup Fish\n\n,Underworld,V2,"Mar 1, 1999",79.0,\n8.8\n,Electronic,The third proper LP from the second incarnatio...,\nJoris\n,"Oct 9, 2007",10.0,"\n Underworld's best by far, yes I own Dubnoba..."
84534,10295,1,\n\nBeaucoup Fish\n\n,Underworld,V2,"Mar 1, 1999",79.0,\n8.8\n,Electronic,The third proper LP from the second incarnatio...,\nJenB\n,"Aug 21, 2007",10.0,"\n Like a good wine, gets better and better. I..."


In [20]:
df = df[df['title'].notna()]

In [21]:
df = df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])

In [22]:
def dropn(x):
    return x.replace('\n', '')

In [23]:
df['title'] = df['title'].apply(dropn)
df['user_score'] = df['user_score'].apply(dropn)
df['name'] = df['name'].apply(dropn)
df['review'] = df['review'].apply(dropn)

In [24]:
df['release_date'] = pd.to_datetime(df['release_date'])
df['date'] = pd.to_datetime(df['date'])

In [25]:
df = df.sort_values(['genre', 'release_date'], ascending = (True,False)).reset_index(drop = True)

In [27]:
df.head()

,Unnamed: 0.1,title,artist,label,release_date,metascore,user_score,genre,summary,name,date,rating,review
0,65592,Continuum,John Mayer,Sony,2006-09-12,67.0,8.9,Adult Alternative,The singer-songwriter's first album in three y...,ibadukefan,2014-02-02,10.0,This is John Mayer in the zone. This is where...
1,65593,Continuum,John Mayer,Sony,2006-09-12,67.0,8.9,Adult Alternative,The singer-songwriter's first album in three y...,ToddW,2006-09-27,1.0,"I give Little, Good John kudos for at least t..."
2,65594,Continuum,John Mayer,Sony,2006-09-12,67.0,8.9,Adult Alternative,The singer-songwriter's first album in three y...,ChristopherG.,2007-08-01,3.0,John Mayer... oh John Mayer. A talented blues...
3,65595,Continuum,John Mayer,Sony,2006-09-12,67.0,8.9,Adult Alternative,The singer-songwriter's first album in three y...,jfrotylpe532,2012-12-21,8.0,John Mayer brings a great sounding album as a ...
4,65596,Continuum,John Mayer,Sony,2006-09-12,67.0,8.9,Adult Alternative,The singer-songwriter's first album in three y...,ErinY,2006-09-12,10.0,It is great to have John Mayer back. This alb...


In [28]:
def clean_rev(text):
    text = text.str.replace("<br/>", "")
    text = text.str.replace("'", '')
    text = text.str.replace("-", '')
    text = text.str.replace('(<a).*(>).*(</a>)', '')
    text = text.str.replace('&amp', '')
    text = text.str.replace('&gt', '')
    text = text.str.replace('&lt', '')
    text = text.str.replace('\xa0', ' ')
    text = text.str.replace('[^\w\s]', ' ')
    text = text.str.replace('[0-9]', ' ')
    text = text.str.lower() 
    return text


In [29]:
df['clean_review'] = clean_rev(df['review'])

## 2. Preprocessing & Feature Engineering

In [30]:
stopwords_list=stopwords.words('english')+list(string.punctuation)+['album','albums', 'songs', 'song', 'music', 'like', 'one']

In [31]:
df['clean_review'] = df['clean_review'].apply(lambda text_list: ' '.join([x for x in text_list.split() if x not in stopwords_list]))

In [32]:
df['length'] = df['review'].astype(str).apply(len)
df['word_count'] = df['review'].apply(lambda x: len(str(x).split()))

Classifying review sentiment using Vader

In [33]:
analyzer = SentimentIntensityAnalyzer()


In [34]:
df['sentiment'] = [analyzer.polarity_scores(x)['compound'] for x in df['clean_review']]
df['negative'] = [analyzer.polarity_scores(x)['neg'] for x in df['clean_review']]
df['neutral'] = [analyzer.polarity_scores(x)['neu'] for x in df['clean_review']]
df['positive'] = [analyzer.polarity_scores(x)['pos'] for x in df['clean_review']]

In [35]:
df['sent_class'] = np.where(df['sentiment'] >= 0.05, 1, df['sentiment'])
df['sent_class'] = np.where((df['sentiment'] > -0.05) & (df['sentiment'] < 0.05), 0, df['sent_class'])
df['sent_class'] = np.where(df['sentiment'] <= -0.05, -1, df['sent_class'])

In [36]:
df['rating_sent'] = np.where(df['rating'] >= 7, 1, df['rating'])
df['rating_sent'] = np.where((df['rating'] > 3) & (df['rating'] < 7), 0, df['rating_sent'])
df['rating_sent'] = np.where(df['rating'] <= 3, -1, df['rating_sent'])

In [24]:
df.genre.value_counts()

Rock                 25773
Indie                17871
Rap                  14975
Pop                  12165
Electronic            5419
R&B                   2808
Country               1637
Dance                 1443
Folk                   442
Alternative            383
Latin                  193
Adult Alternative      151
Jazz                   139
World                  121
Bluegrass               77
Soundtrack              75
Metal                   59
Reggae                  58
Punk                    51
Experimental            42
Comedy                  40
Alt-Country             34
Blues                   30
House                   29
Soul                    28
Singer-Songwriter        5
Live                     3
Adult-Alternative        2
Singer/Songwriter        1
Name: genre, dtype: int64

Lemmatizing with spaCy

In [40]:
sp = spacy.load('en_core_web_md')
lookups = Lookups()
lemm = Lemmatizer(lookups)

In [41]:
def lem_function(text):
    dummy = []
    for word in sp(text):
        dummy.append(word.lemma_)
    return ' '.join(dummy)

In [42]:
tqdm.pandas()

/Users/danielmocombe/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [43]:
df['sp_lm'] = df['clean_review'].progress_apply(lambda x: lem_function(x))


  2%|▏         | 1578/84054 [00:17<11:43, 117.22it/s]


  4%|▍         | 3507/84054 [00:33<09:38, 139.13it/s]


  6%|▌         | 5197/84054 [00:49<16:40, 78.79it/s]


  8%|▊         | 6702/84054 [01:06<14:39, 87.92it/s]


 10%|█         | 8615/84054 [01:22<09:02, 139.07it/s]


...

 92%|█████████▏| 77682/84054 [12:06<00:48, 130.87it/s]


 95%|█████████▍| 79587/84054 [12:22<00:42, 105.87it/s]


 97%|█████████▋| 81255/84054 [12:38<00:25, 111.26it/s]


 99%|█████████▉| 83042/84054 [12:53<00:09, 107.98it/s]


100%|██████████| 84054/84054 [13:03<00:00, 107.28it/s]


In [44]:
df['sp_lm'] = df['sp_lm'].progress_apply(lambda x: x.replace('-PRON-', ' '))


100%|██████████| 84054/84054 [00:00<00:00, 316602.84it/s]


In [50]:
df.head()

,title,artist,label,release_date,metascore,user_score,genre,summary,name,date,...,clean_review,length,word_count,sentiment,negative,neutral,positive,sent_class,rating_sent,sp_lm
0,Continuum,John Mayer,Sony,2006-09-12,67.0,8.9,Adult Alternative,The singer-songwriter's first album in three y...,ibadukefan,2014-02-02,...,john mayer zone lives kind making rest career ...,441,83,-0.3761,0.154,0.728,0.118,-1.0,1.0,john mayer zone live kind make rest career kno...
1,Continuum,John Mayer,Sony,2006-09-12,67.0,8.9,Adult Alternative,The singer-songwriter's first album in three y...,ToddW,2006-09-27,...,give little good john kudos least turning ligh...,575,102,-0.3651,0.222,0.576,0.202,-1.0,-1.0,give little good john kudos least turn light s...
2,Continuum,John Mayer,Sony,2006-09-12,67.0,8.9,Adult Alternative,The singer-songwriter's first album in three y...,ChristopherG.,2007-08-01,...,john mayer oh john mayer talented bluesguitari...,653,117,0.9371,0.057,0.678,0.266,1.0,-1.0,john mayer oh john mayer talented bluesguitari...
3,Continuum,John Mayer,Sony,2006-09-12,67.0,8.9,Adult Alternative,The singer-songwriter's first album in three y...,jfrotylpe532,2012-12-21,...,john mayer brings great sounding matter fact w...,108,20,0.7964,0.000,0.497,0.503,1.0,1.0,john mayer bring great sounding matter fact wa...
4,Continuum,John Mayer,Sony,2006-09-12,67.0,8.9,Adult Alternative,The singer-songwriter's first album in three y...,ErinY,2006-09-12,...,great john mayer back definitely best really s...,123,22,0.9001,0.000,0.389,0.611,1.0,1.0,great john mayer back definitely good really s...
